In [10]:
!pip install -r requirements.txt

In [1]:
# import required libraries
import hana_ml
import pandas as pd
import requests
from hana_ml.dataframe import ConnectionContext

In [8]:
# Connect to SAP HANA Cloud
# The JSON Document Store has to be enabled.
# This does NOT work with SAP HANA Cloud free tier or trial!

# Connect using secure store
# cc = ConnectionContext(userkey='[userkey]', encrypt=True)

host = '[YourHostName]' # e.g. somecharacters.hanacloud.ondemand.com
port = 443
user = '[YourUser]' # e.g. DBADMIN
password = '[YourUserPassword]'
cc= ConnectionContext(
    address=host, 
    port=port, 
    user=user, 
    password=password, 
    encrypt='true'
    )
schema="DAT285"
print('HANA version:', cc.hana_version())
print('hana-ml version:', hana_ml.__version__)
print('pandas version:', pd.__version__)

HANA version: 4.00.000.00.1695725494 (fa/CE2023.28)
hana-ml version: 2.18.23091401
pandas version: 2.0.3


In [4]:
# All car ways
# way["highway"]["area"!~"yes"]["highway"!~"abandoned|bridleway|bus_guideway|construction|corridor|cycleway|elevator|escalator|footway|path|pedestrian|planned|platform|proposed|raceway|service|steps|track"]["motor_vehicle"!~"no"]["motorcar"!~"no"]["service"!~"alley|driveway|emergency_access|parking|parking_aisle|private"]

# Orlando area (28.365266048079008,-81.54412854399905, 28.62735114041908, -81.25956141698434)
overpass_query = """
    [out:json];
    (
    way(28.365266048079008,-81.54412854399905, 28.62735114041908, -81.25956141698434)["highway"]["area"!~"yes"]["highway"!~"abandoned|bridleway|bus_guideway|construction|corridor|cycleway|elevator|escalator|footway|path|pedestrian|planned|platform|proposed|raceway|service|steps|track"]["motor_vehicle"!~"no"]["motorcar"!~"no"]["service"!~"alley|driveway|emergency_access|parking|parking_aisle|private"];
    );
    out body;
    >;
    out skel qt;
"""
overpass_url = "http://overpass-api.de/api/interpreter"
response = requests.get(overpass_url, params={'data': overpass_query})
data = response.json()

In [9]:
# The overpass API resturns JSON which we can store in the HANA Document Store.
from hana_ml.docstore import create_collection_from_elements
coll = create_collection_from_elements(
    connection_context = cc,
    schema = schema,
    collection_name = 'C_STREET_NETWORK',
    elements = data["elements"], 
    drop_exist_coll = True
    )